In [ ]:
!pip install ase
import numpy as np
import warnings
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.animation import FuncAnimation
from ase.io import read
import IPython.display as display
from IPython.display import HTML
warnings.filterwarnings("ignore",category=RuntimeWarning)

In [ ]:
# Function to calculate Lennard-Jones potential
def lj_potential(r, epsilon, sigma):
    return 4 * epsilon * ((sigma / r)**12 - (sigma / r)**6)
# Function to calculate distance between two points
def distance(position1,position2):
    r_diff=position1-position2
    r_pbc=r_diff
    r2=np.sqrt(np.dot(r_pbc,r_pbc))
    return r2

# Task 1

In [ ]:
def total_energy(positions, num_particles,sigma, epsilon, r_cut):
    energy = 0.0
    for i in range(num_particles):
        for j in range(i + 1, num_particles):
            r = distance(positions[i],positions[j])
            if r < r_cut:
                energy += lj_potential(r, epsilon,sigma)
    return energy

In [ ]:
# Read initial positions from XYZ file
def read_xyz(file):
    with open(file, 'r') as f:
        lines = f.readlines()
        num_particles = int(lines[0])
        atom_data = [line.split() for line in lines[2:2 + num_particles]]
        atom_types = [data[0] for data in atom_data]
        positions = np.array([[float(x) for x in data[1:4]] for data in atom_data])
    return atom_types, positions
# Write positions to XYZ file
def write_xyz(file, atom_types, positions, step, energy):
    num_particles = len(atom_types)
    with open(file, 'a') as f:
        f.write(f"{num_particles}\n")
        f.write(f"Step {step}, Energy = {energy}\n")
        for atom_type, pos in zip(atom_types, positions):
            f.write(f"{atom_type} {pos[0]:.6f} {pos[1]:.6f} {pos[2]:.6f}\n")

In [ ]:
# Lennard-Jones potential parameters for Argon
sigma = 3.4  # in Angstroms
epsilon = 0.238  # in kcal/mol

# Simulation parameters
temperature = 200.0  # Temperature in Kelvin
r_cut = 2.5 * sigma  # Cutoff distance, scaled by sigma
num_steps = 50  # Number of Monte Carlo steps
dr_max = 0.1 * sigma  # Maximum displacement, scaled by sigma

In [ ]:
input_file = 'argons.xyz'
output_file = 'trajectory_out.xyz'
atom_types, positions = read_xyz(input_file)
num_particles = len(atom_types)


# Task 2

In [ ]:
#Monte Carlo loop
k_b=1.38064852e-23/1.9872041e-3 #in Kcal/mol/K
for step in range(1, num_steps + 1):
    i = np.random.randint(num_particles)
    old_position = positions[i].copy()
    old_energy = total_energy(positions, num_particles, sigma, epsilon, r_cut)

    # Trial move
    positions[i] += dr_max * (np.random.rand(3) - 0.5)

    new_energy = total_energy(positions, num_particles, sigma, epsilon, r_cut)
    delta_energy = new_energy - old_energy

    # Metropolis criterion
    if delta_energy<0 or np.random.rand() < np.exp(-delta_energy / (k_b * temperature )):
        energy = new_energy
        status='accepted'
    else:
        positions[i] = old_position  # Reject move
        status='rejected'

    # Write positions to the output file after each step
    write_xyz(output_file, atom_types, positions, step, energy)

    print(f'Step:{step}, Move {status}, Energy={energy}')

In [ ]:
plt.ion()

# Function to read the XYZ file
def read_xyz_file(filename):
    atoms = read(filename, index=':')  # Reads all frames
    return atoms

def update(num, atoms, graph, ax):
    ax.clear()  # Clear previous frame
    ax.set_xlim([atoms[num].positions[:, 0].min() - 1, atoms[num].positions[:, 0].max() + 1])
    ax.set_ylim([atoms[num].positions[:, 1].min() - 1, atoms[num].positions[:, 1].max() + 1])
    ax.set_zlim([atoms[num].positions[:, 2].min() - 1, atoms[num].positions[:, 2].max() + 1])
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    graph = ax.scatter(atoms[num].positions[:, 0], atoms[num].positions[:, 1], atoms[num].positions[:, 2], c='blue', marker='o')
    return graph,

filename = 'trajectory_out.xyz'

# Read the XYZ file
trajectory = read_xyz_file(filename)

# Setup the figure and axis for 3D plotting
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

# Create animation
ani = FuncAnimation(fig, update, frames=len(trajectory), fargs=(trajectory, None, ax), blit=False)
html = HTML(ani.to_jshtml())
display.display(html)

In [ ]:
def read_xyz_multi_frame(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()
    
    frames = []
    i = 0
    while i < len(lines):
        num_particles = int(lines[i].strip())
        positions = []
        for line in lines[i+2:i+2+num_particles]:
            _, x, y, z = line.split()
            positions.append([float(x), float(y), float(z)])
        positions = np.array(positions)
        frames.append(positions)
        i += num_particles + 2
    
    return frames, num_particles

# Task 3
#### Calculate average potential energy 

In [ ]:
frames, num_particles = read_xyz_multi_frame(output_file)
# Calculate the total potential energy for each frame
energies = []
for positions in frames:
    total_potential_energy = total_energy(positions, num_particles, sigma, epsilon, r_cut)
    energies.append(total_potential_energy)

# Calculate the average potential energy per frame
average_potential_energy_per_frame = np.mean(energies)

print(f"Average potential energy per frame: {average_potential_energy_per_frame:.4f}")

# Task 4
#### i. Complete closest_distances function to calculate closest distance
#### ii. Calculate closest distances for all frames
#### iii. Plot distribution of closest distances using sns.distplot

In [ ]:
def closest_distances(positions, num_particles):
    closest_dist = []
    for i in range(num_particles):
        min_dist = np.inf
        for j in range(num_particles):
            if i != j:
                dist = distance(positions[i], positions[j])
                if dist < min_dist:
                    min_dist = dist
        closest_dist.append(min_dist)
    return closest_dist

In [ ]:
# Calculate the closest distances for all frames
all_closest_distances = []
for positions in frames:
    closest_dist = closest_distances(positions, num_particles)
    all_closest_distances.extend(closest_dist)

In [ ]:
# Plot the distribution of the closest distances
import seaborn as sns
sns.distplot(all_closest_distances)
plt.xlabel('r')
plt.ylabel('Density')
plt.title('Distribution of Closest Distances in a Cluster')
plt.show()